In [31]:
from google.colab import drive 
drive.mount('/content/drive')

# Tensorflow 2.x is used 
%tensorflow_version 2.x

#IMPORTS
import os
from google.colab import files
import matplotlib.pyplot as plt
import math
import numpy as np
from PIL import Image
import cv2 as cv
import random
from skimage.color import rgb2gray

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Sequential
from keras.layers import Dense,Activation,Dropout,Flatten, Conv2D,MaxPooling2D
import keras.backend as K

# Training is done on a GPU
assert len(tf.config.list_physical_devices('GPU')) > 0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data_dir = '/content/drive/MyDrive/Bismuth_Data_Processed/'

In [3]:
for file in os.scandir(data_dir):
  print(file.path)

/content/drive/MyDrive/Bismuth_Data_Processed/Bismuth Scan Notes COMPLETE - Useful scans.csv
/content/drive/MyDrive/Bismuth_Data_Processed/processed data scan100.txt
/content/drive/MyDrive/Bismuth_Data_Processed/Bismuth Scan Notes COMPLETE - Scans by fluence.csv
/content/drive/MyDrive/Bismuth_Data_Processed/Bismuth Scan Notes COMPLETE - Useful scans.tsv
/content/drive/MyDrive/Bismuth_Data_Processed/MONEYSHOT
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-25
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-21
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-17
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-16
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-12
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-18
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-19
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-22
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-24
/content/drive/MyDrive/Bismuth_Data_Processed/2021-02-09
/content/drive/MyD

In [4]:
def process_data(data_dir):
  data_array = []
  for subdirectories, directories, files in os.walk(data_dir):
    for file_name in files:
      file_loc = subdirectories + os.path.sep + file_name
      
      if file_loc.endswith(".jpg"):      
        photo = Image.open(file_loc)
        #bw_photo_arr = rgb2gray(np.array(photo))
        photo_arr = np.array(photo)
        data_array.append([photo_arr,file_loc])
        #print(len(data_array))
  random.shuffle(data_array)
  print("Done! Added {} images".format(len(data_array)))
  return data_array


In [5]:
data_array = process_data(data_dir)

Done! Added 7962 images


In [6]:
photos,file_names = zip(*data_array)


In [7]:
i = 0
labels = []
usable_photos = []

for file_num in range(0,len(file_names)):
  processed_file_name = file_names[file_num].lower().strip().replace(' ','').split('bismuth_data_processed')[1]
  """
  print(processed_file_name)
  scan_num = processed_file_name.split('scan')[3].split('_gl')[0]
  time = processed_file_name.split('scan',-1)[3].split('_gl')[1].split('_fs_')[0]
  power = processed_file_name.split('scan')[3].split('_gl')[1].split('_fs_')[1].split('_')[0]
  print(scan_num)
  print(time)
  print(power)
  """
  if 'on' in processed_file_name:
    labels.append([0,1])
    usable_photos.append(photos[file_num])
  elif 'off' in processed_file_name:
    labels.append([1,0])
    usable_photos.append(photos[file_num])
#print(labels[0].shape)
labels_array = np.array(labels).reshape(-1,2)
print(len(labels_array))

photos_array = np.array(usable_photos).reshape(-1,150,150,3)
print(photos_array.shape)

7614
(7614, 150, 150, 3)


In [46]:
# Defines the neural network that will be trained on the player's data 
# Returns uncompiled network
def build_resnet_model(lr):
  """
  Transfer learning: a ResNet50 trained on imagenet is used with its last 4 layers removed.
  Trainable fully connected layers are added on top of the imagenet conv layers
  Dropout was initially used but was later removed for performance reasons
  https://towardsdatascience.com/pitfalls-with-dropout-and-batchnorm-in-regression-problems-39e02ce08e4d
  """
  resnet = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

  for layer in resnet.layers[:-4]:
    layer.trainable = False

  # Prints network details for debugging
  """
  for layer in resnet.layers:
    print(layer, layer.trainable)
  """

  model = Sequential()
  model.add(resnet)
  # model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(500, activation='tanh'))
  # model.add(Dropout(0.5))
  model.add(Dense(200, activation='tanh'))
  # model.add(Dropout(0.5))
  model.add(Dense(50, activation='tanh'))
  # model.add(Dropout(0.5))
  model.add(Dense(10, activation='tanh'))
  # model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate = lr), metrics=['accuracy'])
  return model

def build_alexnet():
  #Instantiate an empty model
  model = Sequential()
  # 1st Convolutional Layer
  model.add(Conv2D(filters=96, input_shape=(150,150,3), kernel_size=(10,10), strides=(2,2), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
  # 2nd Convolutional Layer
  model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
  # 3rd Convolutional Layer
  model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # 4th Convolutional Layer
  model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # 5th Convolutional Layer
  model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
  model.add(Activation('relu'))
  # Max Pooling
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
  # Passing it to a Fully Connected layer
  model.add(Flatten())
  # 1st Fully Connected Layer
  model.add(Dense(4096, input_shape=(224*224*3,)))
  model.add(Activation('relu'))
  # Add Dropout to prevent overfitting
  model.add(Dropout(0.6))
  # 2nd Fully Connected Layer
  model.add(Dense(4096))
  model.add(Activation('relu'))
  # Add Dropout
  model.add(Dropout(0.6))
  # 3rd Fully Connected Layer
  model.add(Dense(1000))
  model.add(Activation('relu'))
  # Add Dropout
  model.add(Dropout(0.6))
  # Output Layer
  model.add(Dense(2))
  model.add(Activation('softmax'))
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate = 0.01), metrics=['accuracy'])
  return model

def build_cnn_model():
  cnn_model = keras.Sequential([
      keras.layers.Conv2D(filters=18,input_shape=(150,150,3),kernel_size=(5,5),activation=tf.nn.relu),           
      keras.layers.MaxPool2D(pool_size=(2, 2)),
      keras.layers.Conv2D(filters=36,kernel_size=(5,5),activation=tf.nn.relu), 
      keras.layers.MaxPool2D(pool_size=(2, 2)),
      keras.layers.Flatten(),
      keras.layers.Dense(128, activation=tf.nn.relu),
      keras.layers.Dropout(0.6),
      keras.layers.Dense(2, activation=tf.nn.softmax)
  ])
  cnn_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
  return cnn_model

In [ ]:

#model = build_resnet_model(0.0008)
model = build_cnn_model()
accuracy_values = []
validation_accuracy_values= []
loss_values = []
validation_loss_values = []

EPOCHS = 50

#Training Neural Net
for i in range(0,EPOCHS):
  print('----- --------STARTING EPOCH {}-------------'.format(i))

  history = model.fit(photos_array,labels_array,batch_size=30,epochs=1,validation_split=0.1)

  #Storing accuracy and loss values
  accuracy_values.append(history.history['accuracy'][0])
  validation_accuracy_values.append(history.history['val_accuracy'][0])
  loss_values.append(history.history['loss'][0])
  validation_loss_values.append(history.history['val_loss'][0])

#Plotting NN performance
f1 = plt.figure()
f2 = plt.figure()
ax1 = f1.add_subplot(111)

ax1.plot(accuracy_values, "-b",label = "train accuracy")
ax1.plot(validation_accuracy_values, "-r",label = "test accuracy")
ax1.legend()
ax2 = f2.add_subplot(111)
ax2.plot(loss_values, "-b",label = "train loss")
ax2.plot(validation_loss_values, "-r",label = "test loss")
ax2.legend()
plt.show()


----- --------STARTING EPOCH 0-------------
229/229 [==============================] - 3s 11ms/step - loss: 0.9949 - accuracy: 0.5445 - val_loss: 0.6412 - val_accuracy: 0.6102
----- --------STARTING EPOCH 1-------------
229/229 [==============================] - 2s 11ms/step - loss: 0.6371 - accuracy: 0.5933 - val_loss: 0.6245 - val_accuracy: 0.6115
----- --------STARTING EPOCH 2-------------
229/229 [==============================] - 2s 11ms/step - loss: 0.6557 - accuracy: 0.5736 - val_loss: 0.6520 - val_accuracy: 0.5892
----- --------STARTING EPOCH 3-------------
229/229 [==============================] - 2s 11ms/step - loss: 0.6461 - accuracy: 0.5849 - val_loss: 0.6434 - val_accuracy: 0.5932
----- --------STARTING EPOCH 4-------------
229/229 [==============================] - 2s 11ms/step - loss: 0.6271 - accuracy: 0.5822 - val_loss: 0.6419 - val_accuracy: 0.5997
----- --------STARTING EPOCH 5-------------
229/229 [==============================] - 2s 11ms/step - loss: 0.6301 - acc